# WorkFlow
### Imports
### Load the data
### Cleanning
### FE
### Data.corr()
### Analytics
### Preproccessing
### Decomposition
### Feature Selection
### Modelling
### Random Search
### Gird Search

## Imports

In [ ]:
import random
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import torch,torchvision
from torch.nn import *
from torch.optim import *
# Preproccessing
from sklearn.preprocessing import (
    StandardScaler,
    RobustScaler,
    MinMaxScaler,
    MaxAbsScaler,
    OneHotEncoder,
    Normalizer,
    Binarizer
)
# Decomposition
from sklearn.decomposition import PCA
from sklearn.decomposition import KernelPCA
# Feature Selection
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import RFECV
from sklearn.feature_selection import SelectFromModel
# Model Eval
from sklearn.compose import make_column_transformer
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score,train_test_split
from sklearn.metrics import mean_absolute_error,mean_squared_error,accuracy_score,precision_score,f1_score,recall_score
# Models
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LogisticRegression,LogisticRegressionCV
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor,AdaBoostRegressor,VotingRegressor,BaggingRegressor,RandomForestRegressor
from sklearn.svm import SVR
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import ExtraTreesRegressor
from catboost import CatBoost,CatBoostRegressor
from xgboost import XGBRegressor,XGBRFRegressor
from flaml import AutoML
# Other
import pickle
import wandb

PROJECT_NAME = 'House-Prices-Advanced-Regression-Techniques-V9'
device = 'cuda'
np.random.seed(21)
random.seed(21)
torch.manual_seed(21)

### Funtions

In [ ]:
def make_submission(model):
    pass

In [ ]:
def valid(model,X,y,valid=False):
    preds = model.predict(X)
    if valid:
        results = {
            'val mean_absolute_error':mean_absolute_error(y_true=y,y_pred=preds),
            'val mean_squared_error':mean_squared_error(y_true=y,y_pred=preds),
        }
    else:
        results = {
            'mean_absolute_error':mean_absolute_error(y_true=y,y_pred=preds),
            'mean_squared_error':mean_squared_error(y_true=y,y_pred=preds),
        }
    return results

In [ ]:
def train(model,X_train,X_test,y_train,y_test,name):
    wandb.init(project=PROJECT_NAME,name=name)
    model.fit(X_train,y_train)
    wandb.log(valid(model,X_train,y_train))
    wandb.log(valid(model,X_test,y_test,True))
    make_submission(model)
    return model

In [ ]:
def object_to_int(data,col):
    data_col = data[col].to_dict()
    idx = -1
    labels_and_int_index = {}
    for data_col_vals in data_col.values():
        if data_col_vals not in labels_and_int_index.keys():
            idx += 1
            labels_and_int_index[data_col_vals] = idx
    new_data = []
    for data_col_vals in data_col.values():
        new_data.append(labels_and_int_index[data_col_vals])
    data[col] = new_data
    return data,idx,labels_and_int_index,new_data

In [ ]:
def fe(data,col,quantile_max_num=0.95,quantile_min_num=0.05):
    max_num = data[col].quantile(quantile_max_num)
    min_num = data[col].quantile(quantile_min_num)
    data = data[data[col] < max_num]
    data = data[data[col] > min_num]
    return data

In [ ]:
def decomposition(X,pca=False,kernal_pca=False):
    if pca:
        pca = PCA()
        X = pca.fit_transform(X)
    if kernal_pca:
        kernal_pca = KernelPCA()
        X = kernal_pca.fit_transform(X)
    return X

In [ ]:
def feature_selection_prep_data(model,X,y,select_from_model=False,variance_threshold=False,select_k_best=False,rfecv=False):
    if select_from_model:
        transform = SelectFromModel(estimator=model.fit(X, y))
        X = transform.transform(X)
    if variance_threshold:
        transform = VarianceThreshold()
        X = transform.fit_transform(X)
    if select_k_best:
        X = SelectKBest(chi2, k='all').fit_transform(X, y)
    if rfecv:
        X = RFECV(model, step=1, cv=5).fit(X, y)
        X = X.transform(X)
    return X

In [ ]:
def prep_data(X,transformer):
    mct = make_column_transformer(
        (transformer,list(X.columns)),
        remainder='passthrough'
    )
    X = mct.fit_transform(X)
    return X

## Load the data

In [ ]:
data = pd.read_csv('./data/train.csv')

In [ ]:
preproccessings = [StandardScaler,RobustScaler,MinMaxScaler,MaxAbsScaler,OneHotEncoder,Normalizer,Binarizer]

In [ ]:
models = [
    ['KNeighborsRegressor',KNeighborsRegressor],
    ['LogisticRegression',LogisticRegression],
    ['LogisticRegressionCV',LogisticRegressionCV],
    ['DecisionTreeRegressor',DecisionTreeRegressor],
    ['GradientBoostingRegressor',GradientBoostingRegressor],
    ['AdaBoostRegressor',AdaBoostRegressor],
    ['RandomForestRegressor',RandomForestRegressor],
    ['BaggingRegressor',BaggingRegressor],
    ['GaussianNB',GaussianNB],
    ['ExtraTreesRegressor',ExtraTreesRegressor],
    ['CatBoost',CatBoost],
    ['CatBoostRegressor',CatBoostRegressor],
    ['XGBRegressor',XGBRegressor],
    ['XGBRFRegressor',XGBRFRegressor],
    ['ExtraTreesRegressor',ExtraTreesRegressor],
]

## Cleanning the data

In [ ]:
X = data.drop('SalePrice',axis=1)
y = data['SalePrice']

In [ ]:
str_cols = []
int_cols = []

In [ ]:
for col_name,num_of_missing_rows,dtype in zip(list(X.columns),X.isna().sum(),X.dtypes):
    if dtype == object:
        str_cols.append(col_name)
    else:
        int_cols.append(col_name)

In [ ]:
for str_col in str_cols:
    X,idx,labels_and_int_index,new_data = object_to_int(X,str_col)

In [ ]:
X.head()

In [ ]:
nan_cols = []
for col_name,num_of_missing_rows,dtype in zip(list(X.columns),X.isna().sum(),X.dtypes):
    if num_of_missing_rows > 0:
        nan_cols.append(col_name)

In [ ]:
for nan_col in nan_cols:
    X[nan_col].fillna(X[nan_col].median(),inplace=True)

In [ ]:
nan_cols = []
for col_name,num_of_missing_rows,dtype in zip(list(X.columns),X.isna().sum(),X.dtypes):
    if num_of_missing_rows > 0:
        nan_cols.append(col_name)

In [ ]:
train(GradientBoostingRegressor(),X,X,y,y,name='baseline-without-fe')

In [ ]:
X_old = X.copy()

In [ ]:
for col_name in list(X.columns):
    try:
        X = X_old.copy()
        X = fe(X,col_name)
        train(GradientBoostingRegressor(),X,X,y,y,name=f'baseline-with-fe-{col_name}')
    except:
        pass

In [ ]:
X = X_old.copy()

In [ ]:
X_corr = X_old.corr()

In [ ]:
plt.figure(figsize=(12,7))
sns.heatmap(X_corr)